In [3]:
import pandas as pd
# cancer = "HNSC"
cancer = "COAD"
# cancer = "STAD"
# d = pd.read_csv(f"Data/{cancer}/gene_biome_{cancer}.csv", index_col=0)

In [1]:
import loader as load
ge_phylum_stage = load.loadGEOverlappingTCMA("phylum", includeStage=True)

In [14]:
import processor as pr
stad_ge_phylum_stage = ge_phylum_stage[ge_phylum_stage["project"]=="STAD"]
# print(stad_ge_phylum_stage["stage"].values)
stage_dictionary = {
    "I":0,
    "IA":0,
    "IB":0,
    "II":1,
    "IIA":1,
    "IIB":1,
    "III":2,
    "IIIA":2,
    "IIIB":2,
    "IIIC":2,
    "IV":3,
}

def convertStage(stage):
    st = str(stage)[6:]
    return stage_dictionary[st]
stad_ge_phylum_stage["stage"] = stad_ge_phylum_stage.apply(lambda row: convertStage(row.stage) if row.stage==row.stage else row.stage, axis=1)   
print(stad_ge_phylum_stage["stage"].values)
# load.attachTumorStatus(stad_ge_phylum_stage)
x, y = pr.splitData(stad_ge_phylum_stage, "stage")

[ 0.  0.  0.  2.  3.  2.  3.  1.  1.  1.  1.  3.  1.  1.  3.  2.  2. nan
  2.  2.  0. nan  1. nan nan  0.  2.  2. nan  0.  3.  2. nan  3.  2.  2.
  2.  1.  0.  3.  2.  3.  1.  3. nan  1. nan  2. nan  1.  1.  3.  2.  2.
  2.  0.  0.  2.  2.  3.  1.  0.  2.  1.  0.  2.  1. nan nan  1.  0.  2.
  1.  3.  0.  1.  0.  2.  2.  3.  1.  1. nan  1. nan  2.  2.  0.  1.  3.
  0.  1.  0.  3.  2.  1. nan  3.  1.  1.  0.  1.  1.  2.  1.  3.  3.  1.
  0.  0.  1.  0.  1.  3.  1. nan  1.  1.  1.  1.  2.  1.]


C:\Users\Ameno\AppData\Local\Temp\ipykernel_16716\3993372423.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stad_ge_phylum_stage["stage"] = stad_ge_phylum_stage.apply(lambda row: convertStage(row.stage) if row.stage==row.stage else row.stage, axis=1)


: 

In [16]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
model = SVC()
model.fit(x_train, y_train)


SVC()

In [17]:
y_predicted = model.predict(x_test)
accuracy_score(y_test, y_predicted)

0.9512195121951219

In [18]:
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=0)
skf.get_n_splits(x, y)
for train_index, test_index in skf.split(x, y):
    # print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(x_train, y_train)
    y_predicted = model.predict(x_test)
    print(y_predicted)
    print(accuracy_score(y_test, y_predicted))
    print(classification_report(y_test, y_predicted, output_dict=True))

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1]
0.967741935483871
{'0': {'precision': 1.0, 'recall': 0.5, 'f1-score': 0.6666666666666666, 'support': 2}, '1': {'precision': 0.9666666666666667, 'recall': 1.0, 'f1-score': 0.983050847457627, 'support': 29}, 'accuracy': 0.967741935483871, 'macro avg': {'precision': 0.9833333333333334, 'recall': 0.75, 'f1-score': 0.8248587570621468, 'support': 31}, 'weighted avg': {'precision': 0.9688172043010753, 'recall': 0.967741935483871, 'f1-score': 0.9626389648259521, 'support': 31}}
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
0.9032258064516129
{'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 3}, '1': {'precision': 0.9032258064516129, 'recall': 1.0, 'f1-score': 0.9491525423728813, 'support': 28}, 'accuracy': 0.9032258064516129, 'macro avg': {'precision': 0.45161290322580644, 'recall': 0.5, 'f1-score': 0.47457627118644063, 'support': 31}, 'weighted avg': {'precision': 0.8158168574401664, 'recall': 

C:\Users\Ameno\Anaconda3\envs\bacteria\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ameno\Anaconda3\envs\bacteria\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ameno\Anaconda3\envs\bacteria\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Amen